In [7]:
import arcpy

# 取得目前開啟的 ArcGIS Pro 專案
aprx = arcpy.mp.ArcGISProject("CURRENT")

print("專案路徑：", aprx.filePath)              # .aprx 檔案完整路徑
print("預設 gdb：", aprx.defaultGeodatabase)   # 預設的 geodatabase
print("專案資料夾：", aprx.homeFolder)         # 專案所在的資料夾 (home folder)

# 取得目前活動的 Map (通常就是你 ArcGIS Pro 視窗左邊的那個)
m = aprx.activeMap

print("目前 Map 名稱:", m.name)

sr = m.spatialReference
print("圖層空間參考:", sr.name, sr.factoryCode)

專案路徑： D:\ArcGIS\AI_training\AI_training.aprx
預設 gdb： D:\ArcGIS\AI_training\AI_training.gdb
專案資料夾： D:\ArcGIS\AI_training
目前 Map 名稱: KBay
圖層空間參考: WGS_1984_UTM_Zone_4N 32604


In [1]:
#### 以中心點建立矩形

# ==== 先設定輸出檔案 ====
out_file = "north_reef1_plot_8m"

# ==== 輸入參數 ====
lat = -14.645378   # 緯度 
lon = 145.45491  # 經度
side_len = 8    # 邊長 (公尺)

# 空間參考：輸入經緯度 (WGS84)，再投影到合適的平面座標系統 (例如 UTM Zone 51N)
sr_wgs84 = arcpy.SpatialReference(4326)
sr_proj = arcpy.SpatialReference(32755)  # 請依實際區域調整

# 中心點 (經緯度)
center_point = arcpy.PointGeometry(arcpy.Point(lon, lat), sr_wgs84)

# 投影到平面座標系統
center_proj = center_point.projectAs(sr_proj)

# 計算半邊長
half_side = side_len / 2.0
x, y = center_proj.centroid.X, center_proj.centroid.Y

# 建立矩形的 polygon
array = arcpy.Array([
    arcpy.Point(x - half_side, y - half_side),
    arcpy.Point(x - half_side, y + half_side),
    arcpy.Point(x + half_side, y + half_side),
    arcpy.Point(x + half_side, y - half_side),
    arcpy.Point(x - half_side, y - half_side)  # 關閉圖形
])
rect_polygon = arcpy.Polygon(array, sr_proj)

# ==== 輸出 ====
arcpy.CopyFeatures_management(rect_polygon, out_file)

print("矩形建立完成:", out_file)


矩形建立完成: north_reef1_plot_8m


In [2]:
###輸出plot中的網格

import arcpy
import math
import os

arcpy.env.overwriteOutput = True  # 同名覆蓋
arcpy.env.addOutputsToMap = True  # 自動加到地圖

# ====== 參數 ======
in_fc    = "north_reef1_plot_8m"   # 輸入 Polygon (圖層或要素類別)
side_len = 2                            # 每格邊長 (公尺)
out_fc   = "north_reef1_grid_2m" # 輸出 (可用只寫檔名 + 設定 arcpy.env.workspace)

scratch  = arcpy.env.scratchGDB             # 暫存資料放這

# 暫存路徑
fishnet_fc  = os.path.join(scratch, "fishnet_tmp")
clipped_fc  = os.path.join(scratch, "grid_clip_tmp")

# 刪除舊暫存
for tmp in [fishnet_fc, clipped_fc]:
    if arcpy.Exists(tmp):
        arcpy.management.Delete(tmp)

# 1) 取得範圍
ext = arcpy.Describe(in_fc).extent
origin_coord = f"{ext.XMin} {ext.YMin}"
y_axis_coord = f"{ext.XMin} {ext.YMin + 1}"
opp_corner   = f"{ext.XMax} {ext.YMax}"

# 2) 建立 fishnet
arcpy.management.CreateFishnet(
    out_feature_class=fishnet_fc,
    origin_coord=origin_coord,
    y_axis_coord=y_axis_coord,
    cell_width=side_len,
    cell_height=side_len,
    number_rows=None,
    number_columns=None,
    corner_coord=opp_corner,
    labels="NO_LABELS",
    template=in_fc,
    geometry_type="POLYGON"
)

# 3) Clip 到原 Polygon
arcpy.analysis.Clip(fishnet_fc, in_fc, clipped_fc)

# 4) 複製到最終輸出
arcpy.management.CopyFeatures(clipped_fc, out_fc)
print("完成！輸出：", out_fc)

# 5) 清理暫存
for tmp in [fishnet_fc, clipped_fc]:
    if arcpy.Exists(tmp):
        arcpy.management.Delete(tmp)

# 6) 如果 ArcGIS 已經把暫存圖層加到地圖，從 activeMap 移掉它們
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.activeMap
for name in ["fishnet_tmp", "grid_clip_tmp"]:
    for lyr in m.listLayers(name):
        m.removeLayer(lyr)


完成！輸出： north_reef1_grid_2m


In [3]:
import arcpy
import os

arcpy.env.overwriteOutput = True

# === 參數 ===
fc = "north_reef1_grid_2m"  # 輸入：包含多個小矩形的 Polygon 要素類別
seq_field = "Zone"             # 欄位名稱
start_no = 1                  # 起始編號
use_true_centroid = True  # 用 SHAPE@TRUECENTROID；若要 SHAPE@XY 就改 False

# 若欄位不存在就新增；存在則直接覆寫內容
if seq_field not in [f.name for f in arcpy.ListFields(fc)]:
    arcpy.management.AddField(fc, seq_field, "LONG")

# 1) 取質心
rows = []  # (OID, y, x)
centroid_token = "SHAPE@TRUECENTROID" if use_true_centroid else "SHAPE@XY"

with arcpy.da.SearchCursor(fc, ["OID@", centroid_token]) as cur:
    for oid, centroid in cur:
        x, y = centroid
        rows.append((oid, y, x))

# 2) 左下 → 右上排序（先 Y，再 X，最後 OID 防止平手不定序）
rows_sorted = sorted(rows, key=lambda t: (t[1], t[2], t[0]))

# 3) 建立 OID → 序號 對照
oid_to_seq = {}
seq = start_no
for oid, _, _ in rows_sorted:
    oid_to_seq[oid] = seq
    seq += 1

# 4) 回寫（覆寫欄位值）
oid_field = arcpy.Describe(fc).OIDFieldName
with arcpy.da.UpdateCursor(fc, [oid_field, seq_field]) as ucur:
    for row in ucur:
        oid = row[0]
        row[1] = oid_to_seq.get(oid)   # 不管原來有沒有值，直接覆寫
        ucur.updateRow(row)

print(f"完成！欄位 {seq_field} 已依左下→右上編號（覆寫原有值），共 {len(rows_sorted)} 筆。")

完成！欄位 Zone 已依左下→右上編號（覆寫原有值），共 16 筆。
